In [1]:
import logging
import os
import asyncio

from ogmyrag.report_retrieval.report_retrieval import ReportRetrievalManager
from ogmyrag.report_retrieval.retrieval_storage import RetrievalAsyncStorageManager
from ogmyrag.base import PineconeStorageConfig
from ogmyrag.report_scraper.models import ReportType
from ogmyrag.my_logging import configure_logger
from ogmyrag.storage import PineconeStorage
from ogmyrag.report_retrieval.report_chunker import rag_answer_with_company_detection

from dotenv import load_dotenv

retrieval_logger = configure_logger(name='query',log_level=logging.INFO, log_file='logs/query.log')
retrieval_logger.info("\n" + "=" * 80)

load_dotenv(override=True)

mongo_db_uri = os.getenv("MONGO_DB_URI_JJ","")
pinecone_api_key = os.getenv("PINECONE_API_KEY_JJ","")
genai_api_key = os.getenv("GENAI_API_KEY_JJ","")
openai_api_key = os.getenv("OPENAI_API_KEY", "")

INDEX_NAME = "company-disclosures-index"
EMBED_MODEL = "text-embedding-3-small"
DIMENSION = 1536
GENAI_MODEL = "gemini-2.5-pro"
#GENAI_MODEL = "gemini-2.5-flash"
OPENAI_MODEL = "gpt-5-nano"



pinecone_config: PineconeStorageConfig = {
    "index_name": INDEX_NAME,
    "pinecone_api_key": pinecone_api_key,
    "pinecone_environment": "us-east-1",
    "pinecone_cloud": "aws",
    "pinecone_metric": "cosine",
    "pinecone_dimensions": DIMENSION,
    "openai_api_key": openai_api_key,
}

pine = PineconeStorage(
    pinecone_api_key = pinecone_api_key,
    openai_api_key = openai_api_key
)

pine.create_index_if_not_exists(
    index_name = pinecone_config["index_name"],
    dimension = pinecone_config["pinecone_dimensions"],
    metric = pinecone_config["pinecone_metric"],
    cloud = pinecone_config["pinecone_cloud"],
    region = pinecone_config["pinecone_environment"],
)


2025-09-10 01:49:26,809 - query - INFO - 


## Simple User Query

In [2]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "what does cabnet holdings do? elaborate",
    top_k = 10
)

2025-09-09 20:12:36,968 - query - INFO - RAG start | query='what does cabnet holdings do? elaborate' | top_k=10
2025-09-09 20:12:41,710 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-09 20:12:47,873 - query - INFO - Detected company: 'CABNET_HOLDINGS_BERHAD'
2025-09-09 20:12:47,874 - query - INFO - Search query normalized: 'what does cabnet holdings do? elaborate' → 'what does the company do? elaborate'
2025-09-09 20:12:47,875 - query - INFO - Query filter: {'from_company': 'CABNET_HOLDINGS_BERHAD'}
2025-09-09 20:13:19,097 - query - INFO - Final answer: 

## Overview
Based on the provided documents for Cabne

In [3]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "who are the board directors of cabnet?",
    top_k = 20
)

2025-09-09 20:13:36,866 - query - INFO - RAG start | query='who are the board directors of cabnet?' | top_k=20
2025-09-09 20:13:40,071 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-09 20:13:48,200 - query - INFO - Detected company: 'CABNET_HOLDINGS_BERHAD'
2025-09-09 20:13:48,202 - query - INFO - Search query normalized: 'who are the board directors of cabnet?' → 'Who are the board directors of the company?'
2025-09-09 20:13:48,203 - query - INFO - Query filter: {'from_company': 'CABNET_HOLDINGS_BERHAD'}
2025-09-09 20:14:19,559 - query - INFO - Final answer: 

## Overview
According to the 2019 Corporate Inf

In [3]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "who are the competitors of autocount dotcom?",
    top_k = 10
)

2025-09-10 01:51:18,531 - query - INFO - RAG start | query='who are the competitors of autocount dotcom?' | top_k=10
2025-09-10 01:51:21,866 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-10 01:51:31,836 - query - INFO - Detected company: 'AUTOCOUNT_DOTCOM_BERHAD'
2025-09-10 01:51:31,838 - query - INFO - Search query normalized: 'who are the competitors of autocount dotcom?' → 'Who are the competitors of the company?'
2025-09-10 01:51:31,839 - query - INFO - Query filter: {'from_company': 'AUTOCOUNT_DOTCOM_BERHAD'}
2025-09-10 01:52:13,364 - query - INFO - Final answer: 

## Overview
Autocount Group’s competi

In [6]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "who are the directors of vitrox? elaborate about their background and position",
    top_k = 10
)

2025-09-09 20:17:26,871 - query - INFO - RAG start | query='who are the directors of vitrox? elaborate about their background and position' | top_k=10
2025-09-09 20:17:30,235 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-09 20:17:39,205 - query - INFO - Detected company: 'VITROX_CORPORATION_BERHAD'
2025-09-09 20:17:39,206 - query - INFO - Search query normalized: 'who are the directors of vitrox? elaborate about their background and position' → 'who are the directors of the company? elaborate about their background and position'
2025-09-09 20:17:39,207 - query - INFO - Query filter: {'from_company': 'VITROX

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "What is farm fresh management’s strategy for growth over the next 3–5 years?",
    top_k = 5
)

2025-08-27 11:57:27,006 - retrieval - INFO - RAG start | query='What is farm fresh management’s strategy for growth over the next 3–5 years?' | top_k=5
2025-08-27 11:57:27,570 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 11:57:44,240 - retrieval - INFO - Detected company: 'FARM_FRESH_BERHAD'
2025-08-27 11:57:44,243 - retrieval - INFO - Search query normalized: 'What is farm fresh management’s strategy for growth over the next 3–5 years?' → "What is the company's strategy for growth over the next 3–5 years?"
2025-08-27 11:57:44,243 - retrieval - INFO - Query filter: {'from_company': 'FARM_FRESH_BERHAD'}
2025-08-27 11:57:58,769 - retrieval - INFO - Final answer: 

- Product portfolio expansion: Focus on developing and diversifying products (CPG ice cream via the Inside Scoop brand by Aug 2024; expanding into plant-based yoghurt;

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "Who is Joyce Wong Ai May? What is his/her position? Elaborate more",
    top_k = 5
)

2025-08-27 12:07:42,060 - retrieval - INFO - RAG start | query='Who is Joyce Wong Ai May? What is his/her position? Elaborate more' | top_k=5
2025-08-27 12:07:44,310 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:07:48,327 - retrieval - INFO - Detected company: None
2025-08-27 12:07:48,328 - retrieval - INFO - Search query unchanged.
2025-08-27 12:07:48,329 - retrieval - INFO - Query filter: {}
2025-08-27 12:08:02,078 - retrieval - INFO - Final answer: 

- Joyce Wong Ai May is an Independent Non-Executive Director of Edelteq Holdings Berhad; appointed to the Board on 1 August 2022. She is Malaysian, aged 47.

- Education and professional background:
  - Bachelor of Commerce, University of Tasmania (Dec 1998)
  - Accountant by profession; CPA Australia since Jan 2003
  - Member of the Malaysian Institute of Accountants since S

In [2]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "Who is Liew Soung Yue? What is his/her position? Elaborate more",
    top_k = 5
)

2025-09-09 20:21:05,973 - query - INFO - RAG start | query='Who is Liew Soung Yue? What is his/her position? Elaborate more' | top_k=5
2025-09-09 20:21:10,609 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-09 20:21:15,149 - query - INFO - Detected company: None
2025-09-09 20:21:15,150 - query - INFO - Search query unchanged.
2025-09-09 20:21:15,151 - query - INFO - Query filter: {}
2025-09-09 20:21:50,751 - query - INFO - Final answer: 

## Overview
Dr. Liew Soung Yue is the Independent Non-Executive Director on AutoCount DotCom Berhad’s board, appointed on 6 April 2022. He serves as Chairperson of the Nomin

In [3]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "Who are the board of directors of AutoCount? Tell more about their background.",
    top_k = 10
)

2025-09-09 20:21:50,764 - query - INFO - RAG start | query='Who are the board of directors of AutoCount? Tell more about their background.' | top_k=10
2025-09-09 20:21:54,159 - query - INFO - Catalog companies (14): ['AEMULUS_HOLDINGS_BERHAD', 'AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'CENTURY_SOFTWARE_HOLDINGS_BERHAD', 'ECA_INTEGRATED_SOLUTION_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'I-STONE_GROUP_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'RAMSSOL_GROUP_BERHAD', 'SFP_TECH_HOLDINGS_BERHAD', 'TT_VISION_HOLDINGS_BERHAD', 'VETECE_HOLDINGS_BERHAD', 'VITROX_CORPORATION_BERHAD']
2025-09-09 20:22:05,449 - query - INFO - Detected company: 'AUTOCOUNT_DOTCOM_BERHAD'
2025-09-09 20:22:05,451 - query - INFO - Search query normalized: 'Who are the board of directors of AutoCount? Tell more about their background.' → 'Who are the board of directors of the company? Tell more about their background.'
2025-09-09 20:22:05,452 - query - INFO - Query filter: {'from_company': 'AUTOCOUNT_D

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "What are the mission and vision of EDELTEQ?",
    top_k = 5
)

2025-08-27 19:42:23,422 - query - INFO - RAG start | query='What are the mission and vision of EDELTEQ?' | top_k=5
2025-08-27 19:42:25,816 - query - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 19:42:32,845 - query - INFO - Detected company: 'EDELTEQ_HOLDINGS_BERHAD'
2025-08-27 19:42:32,846 - query - INFO - Search query normalized: 'What are the mission and vision of EDELTEQ?' → 'What are the mission and vision of the company?'
2025-08-27 19:42:32,846 - query - INFO - Query filter: {'from_company': 'EDELTEQ_HOLDINGS_BERHAD'}
2025-08-27 19:42:41,156 - query - INFO - Final answer: 

- Vision: "to convert all our customers’ manufacturing ideas into reality; and enhance their manufacturing process & system efficiency."
- Mission: "to bring Value propositions, Innovative solutions and Niche processes to support semiconductor manufacturing needs 

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "What the company CABNET do?",
    top_k = 5
)

2025-08-26 12:05:49,090 - retrieval - INFO - RAG start | query='What the company CABNET do?' | top_k=5
2025-08-26 12:05:49,670 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-26 12:06:00,430 - retrieval - INFO - Detected company: 'CABNET_HOLDINGS_BERHAD'
2025-08-26 12:06:00,431 - retrieval - INFO - Search query normalized: 'What the company CABNET do?' → 'What the company do?'
2025-08-26 12:06:00,432 - retrieval - INFO - Query filter: {'from_company': 'CABNET_HOLDINGS_BERHAD'}
2025-08-26 12:06:05,128 - retrieval - INFO - Final answer: 

Cabnet Holdings Berhad provides building management solutions, including structured cabling works and ELV systems for buildings and facilities, and IT services. Their activities cover design, supply, build, testing and commissioning, plus project management, training, maintenance and aftersales servi

In [ ]:
out = await rag_answer_with_company_detection(
    pine,
    pinecone_config = pinecone_config,
    query = "What is the vision and mission of farm fresh?",
    top_k = 8
)

2025-08-27 12:10:16,796 - retrieval - INFO - RAG start | query='What is the vision and mission of farm fresh?' | top_k=8
2025-08-27 12:10:17,351 - retrieval - INFO - Catalog companies (6): ['AUTOCOUNT_DOTCOM_BERHAD', 'CABNET_HOLDINGS_BERHAD', 'EDELTEQ_HOLDINGS_BERHAD', 'FARM_FRESH_BERHAD', 'ICT_ZONE_ASIA_BERHAD', 'VETECE_HOLDINGS_BERHAD']
2025-08-27 12:10:25,862 - retrieval - INFO - Detected company: 'FARM_FRESH_BERHAD'
2025-08-27 12:10:25,863 - retrieval - INFO - Search query normalized: 'What is the vision and mission of farm fresh?' → 'What is the vision and mission of the company?'
2025-08-27 12:10:25,864 - retrieval - INFO - Query filter: {'from_company': 'FARM_FRESH_BERHAD'}
2025-08-27 12:10:35,477 - retrieval - INFO - Final answer: 

- Vision
  - "TO BE A SUSTAINABLE AND HONEST FOOD COMPANY THAT HAS AN INGRAINED CULTURE OF PLACING THE WELL-BEING OF CONSUMERS FIRST, CULMINATING IN STRONG BRAND LOVE." 

- Mission
  - "Diversifying our product portfolio to capture a wide, addressab